In [1]:
import scanpy as sc
import numpy as np
import csv
import pickle
import matplotlib
import math
from itertools import combinations
import pandas as pd
import matplotlib
from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite
from cdlib import algorithms
from cdlib import NodeClustering
import altair as alt
import sys
sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
if True:  # In order to bypass isort when saving
    import altairThemes

spot_diameter = 89.43 #pixels

Note: to be able to use all crisp methods, you need to install some additional packages:  {'graph_tool', 'wurlitzer', 'bayanpy', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [6]:
def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df

In [2]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# Load cell_barcode
with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)

# Load connected_components (assume already have it)
with open("/Users/victoriagao/local_docs/NEST/stored_variables/filtered_connected_components.pkl", 'rb') as file:
    filtered_connected_components = pickle.load(file)

# Load pathologist's label
data_name = 'PDAC_64630'
if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]

In [3]:
filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
inputFile = '/Users/victoriagao/local_docs/NEST/output/From_Fatema/'+filename_str
df = pd.read_csv(inputFile, sep=",")
csv_record_final = df.values.tolist()
df_column_names = list(df.columns)
csv_record_final = [df_column_names] + csv_record_final

In [4]:
#### i.e. making a list of Dictionary to store matched records for each region
matched_records_set_lib = {}  

for set_index, connected_comp_set in enumerate(filtered_connected_components):
    matched_records = []

    for record in csv_record_final:
        from_cell, to_cell = record[0], record[1]
        if (from_cell in connected_comp_set) and (to_cell in connected_comp_set):
            matched_records.append(record)
    
    # Store matched records for this set index in the dictionary
    matched_records_set_lib[set_index+1] = matched_records

In [7]:
# Calculate Hill number q=1 for all 21 regions

import math

columns = ['from_cell','to_cell','ligand','receptor','attention_score','component','from_id','to_id']

hill_number_data = []

for key, value in matched_records_set_lib.items():
    df_matched = pd.DataFrame(value, columns=columns)
    df_matched_processed = preprocessDf(df_matched)
    ligand_receptor_counts = df_matched_processed['ligand-receptor'].value_counts()
    count_df = pd.DataFrame({'ligand-receptor': ligand_receptor_counts.index, 'count': ligand_receptor_counts.values})
    # print(count_df)
    # Summing
    total_counts = count_df['count'].sum()
    # Calculate the proportions
    count_df['proportion'] = count_df['count'] / total_counts
    # Calculate the Shannon-Wiener Diversity Index (H')
    shannon_entropy = -(count_df['proportion'] * count_df['proportion'].apply(math.log)).sum()
    # Calculate the Hill number (effective number of types) for q = 1
    hill_number_q1 = math.exp(shannon_entropy)
    # Print or display the calculated Shannon-Wiener Diversity Index
    print("For set "+str(key))
    print("Shannon-Wiener Diversity Index (H'): {:.4f}".format(shannon_entropy))
    print("Hill number of order 1: {:.4f}".format(hill_number_q1))

    hill_number_data.append({'Set': key, 'Hill_number_q1': hill_number_q1}) # putting the hill numbers into a list
    
# Create a DataFrame from the collected data
hill_number_df = pd.DataFrame(hill_number_data)
hill_number_dict = hill_number_df.set_index('Set')['Hill_number_q1'].to_dict()

For set 1
Shannon-Wiener Diversity Index (H'): 3.2846
Hill number of order 1: 26.6979
For set 2
Shannon-Wiener Diversity Index (H'): 1.4942
Hill number of order 1: 4.4557
For set 3
Shannon-Wiener Diversity Index (H'): 3.5169
Hill number of order 1: 33.6791
For set 4
Shannon-Wiener Diversity Index (H'): 3.1094
Hill number of order 1: 22.4073
For set 5
Shannon-Wiener Diversity Index (H'): 3.3757
Hill number of order 1: 29.2451
For set 6
Shannon-Wiener Diversity Index (H'): -0.0000
Hill number of order 1: 1.0000
For set 7
Shannon-Wiener Diversity Index (H'): 2.5062
Hill number of order 1: 12.2586
For set 8
Shannon-Wiener Diversity Index (H'): 1.2425
Hill number of order 1: 3.4641
For set 9
Shannon-Wiener Diversity Index (H'): -0.0000
Hill number of order 1: 1.0000
For set 10
Shannon-Wiener Diversity Index (H'): -0.0000
Hill number of order 1: 1.0000
For set 11
Shannon-Wiener Diversity Index (H'): -0.0000
Hill number of order 1: 1.0000
For set 12
Shannon-Wiener Diversity Index (H'): -0.000